### Ray Jennings
### December 2022
### Weighted Random Forests

<br>

<p>In this Jupyter notebook a Random Forest method is modified to use weighted weak learners instead of using majority voting. For each weak learner within the random forest, the accuracy of each weak learner is determined during the validation phase. A weight vector is computed based on the weak learner accuracies. There are different ways to compute the weights for the weighted random forest, but I have settled on:</p>
    
<p style="text-align: center;">
$
\begin{align}
weight=accuracy^{p}
\end{align}
$
</p>

<p style="text-align: center;">or</p>

<p style="text-align: center;">
$
\begin{align}
weight=\left\lvert{\frac{1}{log(accuracy)}}\right\rvert
\end{align}
$
</p>

&nbsp;
<p>(or other method can be used).</p>
    
<p>In the first weight equation if <i>p</i> is equal to 1, then that represents the traditional majority voting random forest. When <i>p</i> equals 2, I feel that is the best overall choice. As the value of <i>p</i> increases, the values within the weight vector decrease. But in addition, the separation between the weights increases which gives more weight to the higher accuracy weak learner trees. The fact that the weight values decrease as the value of <i>p</i> increases is not an issue since the most accurate weak learner will have the highest weight and therefore the greatest impact to the final prediction. The simple fact that the weight value is lower when <i>p</i> = 3 then when <i>p</i> equals 2 does not degrade the function of the weights since all of the weights are lower.
    
The weight vector can be calculated in other ways besides the two examples listed above and the weights are not limited to weight values less than or equal to 1. However, if there is no upper bound on the weights then the prediction values could be negatively altered particularly if one were to let the weight values exceed the maximum integer value. If weight values greater than 1 were used then I would prefer to scale them with something like a min-max scaler.
    
The weighted random forest classifier implementation used here was modified from and based on the radom forest classifier by: Jason Brownlee, PhD at https://machinelearningmastery.com/
    
The code below compares three different Random Forest implementations:</p>

1. Basic Random Forest
2. Weighted Random Forest
3. The Scikit-learn RandomForestClassifer (considered to be the "heavyweight", ie. the best)


The datasets used were all obtained from the University of California, Irivine's Center for Machine Learning and Intelligent Systems.

1. Sonar - Rock or Mine Dataset
2. Palmer Penguins Species
3. Iris Flowers
4. White Wine Attributes
5. Red Wine Attributes

Each of the three tree implemenatations is run on the same permutation of the dataset for 5 k-fold cross-validations with an iteration on the number of trees from 5, 10, 15, 20, 25.

The results from these tests look promising. Frequently, the weighted random forest could beat the accuracy of both of the other trees. There were some ties between either the clasic random forest and the weighted random forest, or between the weighted random forest and scikit-learn RandomForestClassifer. But rarely does the weighted random forest come in last place. The "_best_" indicator is calculated by taking the average of the 5 accuracy scores for each of the k-fold cross-validations. Alternatively, a simple count could be used to determine the winner for that round.

In [1]:
import numpy as np
import pandas as pd  
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import operator
import copy
import math

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

</br>

### Methods to modify fields of datasets

In [2]:
#
# Load a CSV file
#
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset


#
# Convert string column to float
#
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())


#
# Convert string column to integer
#
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]

    return lookup

</br>

### Methods to load each dataset: Rock-Mine, Penguins, Iris, White Wine, Red Wine

In [3]:
#
# Sonar - Rock or Mine Dataset
#
def load_sonar_dataset():

    filename = '../data/sonar.all-data.csv'

    dataset = load_csv(filename)

    # convert string attributes to integers
    for i in range(0, len(dataset[0])-1):
        str_column_to_float(dataset, i)

    # convert class column to integers
    str_column_to_int(dataset, len(dataset[0])-1)
    
    return dataset


#
# Penguins Species Dataset
#
def load_penguins_dataset():
    
    filename = '../data/penguins_lter.csv'

    df = pd.read_csv('penguins_lter.csv', sep=',')
    df.drop(['Individual ID', 'Clutch Completion', 'studyName', 'Sample Number', 'Stage', 'Date Egg', 'Delta 15 N (o/oo)', 'Delta 13 C (o/oo)', 'Comments'], inplace=True, axis=1)
    cols = list(df.columns.values)
    cols.pop(cols.index('Species'))
    cols.pop(cols.index('Region'))
    cols.pop(cols.index('Island'))
    df = df[cols+['Region', 'Island', 'Species']]  # Reorder with target column at end
    df.dropna(axis=0, how='any', inplace=True)
    df.to_csv('penguins.csv', sep=',', index=False, header=False)

    dataset = load_csv('penguins.csv')

    # convert string attributes to floats
    for i in range(0, len(dataset[0])-4):
        str_column_to_float(dataset, i)

    # convert class column to integers
    str_column_to_int(dataset, len(dataset[0])-4)
    str_column_to_int(dataset, len(dataset[0])-3)
    str_column_to_int(dataset, len(dataset[0])-2)
    str_column_to_int(dataset, len(dataset[0])-1)
    
    #print("DATASET")
    #print(dataset)

    return dataset


#
# Iris dataset
#
def load_iris_dataset():
   
    filename = '../data/bezdekIris.csv'
    
    dataset = load_csv(filename)

    # convert string attributes to floats
    for i in range(0, len(dataset[0])-1):
        str_column_to_float(dataset, i)

    # convert class column to integers
    str_column_to_int(dataset, len(dataset[0])-1)
    
    return dataset    


#
# White Wine dataset
#
def load_white_wine_dataset():
    
    dataset = load_csv('../data/winequality-white-adj.csv')

    # convert string attributes to floats
    for i in range(0, len(dataset[0])-1):
        str_column_to_float(dataset, i)

    # convert class column to integers
    str_column_to_int(dataset, len(dataset[0])-1)
    
    return dataset

#
# Red Wine dataset
#
def load_red_wine_dataset():
    
    dataset = load_csv('../data/winequality-red-adj.csv')

    # convert string attributes to floats
    for i in range(0, len(dataset[0])-1):
        str_column_to_float(dataset, i)

    # convert class column to integers
    str_column_to_int(dataset, len(dataset[0])-1)
    
    return dataset
 

</br>

### Random Forest Classifer source code by Jason Brownlee, PhD
### https://machinelearningmastery.com/implement-random-forest-scratch-python/ ###

In [4]:
#
# Split a dataset into k folds
#
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


#
# Calculate accuracy percentage
#
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    val = correct / float(len(actual))
    return val


#
# Split a dataset based on an attribute and an attribute value
#
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right


#
# Calculate the Gini index for a split dataset
#
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
        
    return gini


#
# Select the best split point for a dataset
#
def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}


#
# Create a terminal node value
#
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)


#
# Create child splits for a node or make terminal
#
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)


#
# Build a decision tree
#
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root


#
# Make a prediction with a decision tree
#
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']


#
# Create a random subsample from the dataset with replacement
#
def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample


#
# Make a prediction with a list of bagged trees
#
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    #print(predictions)
    rval = max(set(predictions), key=predictions.count)
    #print(rval)
    return rval


In [5]:
#
# Adjust weights - raise to power p
#
def adjust_weights(weights, p): 
    #for i in range(2, 11):
    w = [x**p for x in weights]
    return w

#
# L2 Normalized weights
#
def adjust_weights_l2norm(weights):
    w = weights / np.linalg.norm(weights)
    return w


#
# Weighted Random Forest
#
def weighted_random_forest(p_val, trees, train, test, actual, max_depth, min_size, sample_size, n_trees, n_features):
    
    predict_arr = [[-1 for i in range(len(trees))] for j in range(len(test))]
    
    tree_count = 0
    num_correct = list()
    for t in trees:
        n_correct = 0
        row_count = 0
        for row in test:
            p = predict(t, row)
            if p == actual[row_count]:
                n_correct = n_correct + 1
            predict_arr[row_count][tree_count] = p
            row_count = row_count + 1
        
        #print("Num correct: " + str(n_correct) + " for tree: " + str(tree_count))
        num_correct.append(n_correct)
        
        #print(predict_arr)
        tree_count = tree_count + 1
     
    #print("Best correct: " + str(num_correct))
    
    max_correct = max(num_correct)
    # print(max_correct)
    best_tree_idx = num_correct.index(max(num_correct))
    ### print("Best tree: " + str(best_tree_idx) + " with " + str(max_correct))
    
    #weights = [(nc / max_correct / len(trees)) for nc in num_correct]
    weights = [(nc / len(test) ) for nc in num_correct]
    print()
    print("Original Tree Weights:")
    print(weights)
    print()
    weights = adjust_weights(weights, p_val)
    print("Adjusted Tree Weights:")
    print(weights)
    print()
    
    new_predictions = []
    
    for q in predict_arr:
        dict = {}
        for i in range(len(trees)):
            
            if q[i] in dict.keys():
                dict[q[i]] = dict[q[i]] + weights[i]
            else:
                dict[q[i]] = weights[i]
                
        max_key = max(dict.items(), key=operator.itemgetter(1))[0]
        new_predictions.append(max_key)
        # print("New prediction: " + str(max_key))
    
    return new_predictions
    

#
# Original Random Forest
#
def random_forest(p_val, trees, train, test, actual, max_depth, min_size, sample_size, n_trees, n_features):
    predictions = [bagging_predict(trees, row) for row in test]
    return predictions


<br>

### Perform evaluation on: _Random Forest_, _Weighted Random Forest_, _scikit-learn's RandomForestClassifer_

In [6]:
#
# Evaluate both random forests using a cross validation split
#
def evaluate_RF_algorithms(p_val, dataset, algorithm1, algorithm2, n_folds, *args):
    
    folds = cross_validation_split(dataset, n_folds)
    
    scores_rf = list()   # Scores from generic RF
    scores_wrf = list()  # Scores from weighted RF
    scores_skl = list()  # Scores from scikit-learn's RFC
    
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        
        #
        # Save a separate copy of the training set for scikit-learn's RandomForestClassifier  
        #   - same train_X and train_y but for for scikit-learn only
        #
        train_X_skl = copy.deepcopy(train_set)
        train_y_skl = [row[-1] for row in train_X_skl]
        
        train_X_mod = list()
        train_y_mod = list()
        for row in train_X_skl:
            row_copy = list(row)
            train_y_mod.append(row_copy[-1])
            del row_copy[-1]
            train_X_mod.append(row_copy)
        
        test_y = [row[-1] for row in fold]
        
        test_X = list()
        for row in fold:
            row_copy = list(row)
            del row_copy[-1]
            test_X.append(row_copy)
            #row_copy[-1] = None
        
        # Create trees
        trees = list()
        for i in range(n_trees):
            sample = subsample(train_set, sample_size)
            tree = build_tree(sample, max_depth, min_size, n_features)
            trees.append(tree)
        
        
        predicted_rf = algorithm1(p_val, trees, train_set, test_X, test_y, *args)
        accuracy_rf = accuracy_metric(test_y, predicted_rf)
        scores_rf.append(accuracy_rf)
        
        predicted_wrf = algorithm2(p_val, trees, train_set, test_X, test_y, *args)
        accuracy_wrf = accuracy_metric(test_y, predicted_wrf)
        scores_wrf.append(accuracy_wrf)
        
        #
        # Test scikiet-learn RFC
        #
        rfc = RandomForestClassifier(n_estimators=n_trees, criterion='gini', max_depth=max_depth, max_features='sqrt', bootstrap=False, oob_score=False, warm_start=False, 
                                     max_samples=None, min_samples_leaf=min_size, n_jobs=1, random_state=0, verbose=0)
        rfc.fit(train_X_mod, train_y_mod)
        predict = rfc.predict(test_X)
        accuracy_skl_rf = np.mean(predict == test_y)
        scores_skl.append(accuracy_skl_rf)
        
        
    return scores_rf, scores_wrf, scores_skl


<br>

# Perform tests....

In [8]:
#
# Test and evaluate the 3 different random forests from the following datasets
#

seed(2)

#
# Pick one dataset at a time to evalutate
#

dataset = load_sonar_dataset()
# dataset = load_penguins_dataset()
# dataset = load_iris_dataset()
# dataset = load_red_wine_dataset()
# dataset = load_white_wine_dataset()


#
# hyper-params
#
n_folds = 5
max_depth = 10
min_size = 1  # For SKL: min_samples_leaf, default=1
sample_size = 1.0  # Ratio of data to build the tree, for SKL: max_samples, default=None == use X.shape[0] samples
n_features = int(sqrt(len(dataset[0])-1))   # For SKL: max_features, default=”sqrt”
folds = cross_validation_split(dataset, n_folds)


#
# Count how many trees in each Random Forest are better than the other Random Forest
#

# Alter this list based on the dataset
for p_val in [1, 2, 3, 4, 5, 10]:
    
    print("\n\n")
    print("***********")
    print("For p = " + str(p_val) + "...")
    print("***********")

    for n_trees in [5, 10, 15, 20, 25]:

        print("For " + str(n_trees) + " trees...")
    
        #
        # Evaluate all 3 Random Forest implementations
        #
        scores_rf, scores_wrf, scores_skl = evaluate_RF_algorithms(p_val, dataset, random_forest, weighted_random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    
    
        print("\n\nRESULTS:")
    
        rf_avg = 0.0
        wrf_avg = 0.0
        skl_avg = 0.0
    
        rf_count = 0
        wrf_count = 0
        for i in range(n_folds):
        
            rf_avg += scores_rf[i]
            wrf_avg += scores_wrf[i]
            skl_avg += scores_skl[i]
        
            print(str(round(scores_rf[i], 5)) + " " + str(round(scores_wrf[i], 5)))
            if str(scores_wrf[i]) == str(scores_rf[i]):
                print("EQUAL")
            elif scores_rf[i] > scores_wrf[i]:
                rf_count += 1
                print("ORIGINAL RF is better")
            else:
                wrf_count += 1
                print("WEIGHTED RF is better")
            
            s = ""
            if rf_count == wrf_count:
                s = "EQUAL"
            elif rf_count > wrf_count:
                s = "ORIGINAL RF is better"
            else:
                s = "WEIGHTED RF is better"
    
        print("\n\n")
        print("********************************************")
        print("** For p=" + str(p_val) + " for " + str(n_trees) + " trees " + s + " **")
        print("********************************************\n")
    
        rf_avg = rf_avg / 5
        wrf_avg = wrf_avg / 5
        skl_avg = skl_avg / 5
    
        print()
              
        print("\nRF Scores:")
        for i in range(0, 5):
              print(str(round(scores_rf[i], 5)))
        print()
        print(" RF Avg: " + str(round(rf_avg, 5)))
              
        print("\nWRF Scores:")
        for i in range(0, 5):
              print(str(round(scores_wrf[i], 5)))
        print()
        print("WRF Avg: " + str(round(wrf_avg, 5)))
    
        print("\nSci-kit Learn Scores:")
        for i in range(0, 5):
              print(str(round(scores_skl[i], 5)))
        print()
        print("SKL Avg: " + str(round(skl_avg, 5)))
        
        print("\n**************************************************************************\n\n\n")
    




***********
For p = 1...
***********
For 5 trees...

Original Tree Weights:
[0.7804878048780488, 0.5365853658536586, 0.8536585365853658, 0.6829268292682927, 0.5609756097560976]

Adjusted Tree Weights:
[0.7804878048780488, 0.5365853658536586, 0.8536585365853658, 0.6829268292682927, 0.5609756097560976]


Original Tree Weights:
[0.7073170731707317, 0.6341463414634146, 0.6829268292682927, 0.8048780487804879, 0.5365853658536586]

Adjusted Tree Weights:
[0.7073170731707317, 0.6341463414634146, 0.6829268292682927, 0.8048780487804879, 0.5365853658536586]


Original Tree Weights:
[0.7804878048780488, 0.8536585365853658, 0.7560975609756098, 0.7317073170731707, 0.7560975609756098]

Adjusted Tree Weights:
[0.7804878048780488, 0.8536585365853658, 0.7560975609756098, 0.7317073170731707, 0.7560975609756098]


Original Tree Weights:
[0.4878048780487805, 0.6097560975609756, 0.6585365853658537, 0.6585365853658537, 0.7560975609756098]

Adjusted Tree Weights:
[0.4878048780487805, 0.6097560975609756, 0.